In [51]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
#library(themis)
options(repr.matrix.max.rows = 10)

ERROR: Error in library(themis): there is no package called ‘themis’


In [50]:
#Add columns
unscaled_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

unscaled_data <- unscaled_data |>
    mutate(class = as_factor(class))
#pulsar_data 

#Class proportions in pulsar dataset (Imbalanced)
num_obs <- nrow(unscaled_data)
pulsar_proportions <- unscaled_data |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(unscaled_data))
#pulsar_proportions

pulsar_split <- initial_split(unscaled_data, prop = 0.75, strata = class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split) 

#Scale data and Upsample to balance data
pulsar_recipe <- recipe(class ~ skew_integrated_profile + skew_dmsnr, data = pulsar_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
#    step_upsample(class, over_ratio = 1, skip = FALSE) |>
    prep()
#pulsar_recipe

pulsar_data <- bake(pulsar_recipe, pulsar_train)

#pulsar_data |>
#  group_by(Class) |>
#  summarize(n = n())
#pulsar_data

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
  set_engine("kknn") |>
  set_mode("classification")
#knn_spec

knn_fit <- workflow() |>
  add_recipe(pulsar_recipe) |>
  add_model(knn_spec) |>
  fit(data = pulsar_train)
#knn_fit

pulsar_test_predictions <- predict(knn_fit, pulsar_test) |>
    bind_cols(pulsar_test)
#pulsar_test_predictions

pulsar_test_predictions |>
  metrics(truth = class, estimate = .pred_class) |>
  filter(.metric == "accuracy")
#pulsar_test_predictions

Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_integrated_profile, stand_dev_integrated_profile, exc_kurtosis...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in step_upsample(step_center(step_scale(recipe(class ~ skew_integrated_profile + : could not find function "step_upsample"
